# Challenge 06 - Agentic AI

In this notebook, you will build a **Research Assistant Agent** using the Microsoft Agent Framework. This agent leverages **Model Context Protocol (MCP)** to connect to live data sources like Microsoft Learn documentation.

Quick tip! To view the Table of Contents for this Notebook in VS Code or within Codespaces, take a look at the "Explorer" tab, expand the "Outline" section.

## 6.1. Setting Up Your Environment

First, install the Microsoft Agent Framework. The `--pre` flag is required while the Agent Framework is in preview.

In [1]:
%pip install agent-framework-azure-ai --pre

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.9/217.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.3/306.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 28.9 MB/s eta 0:00:00
  Attempting uninstall: azure-ai-agents
    Found existing installation: azure-ai-agents 1.2.0b6
    Uninstalling azure-ai-agents-1.2.0b6:
      Successfully uninstalled azure-ai-agents-1.2.0b6

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 6.1.1 Load Environment Variables

Load your Microsoft Foundry project endpoint and model deployment name from the `.env` file.

**NOTE:** These values in your .env file are required to ensure the notebook runs seamlessly. They should already be there if you deployed using the deployment script in Challenge 0.
* AZURE_AI_PROJECT_ENDPOINT must equal your Microsoft Foundry project endpoint
* CHAT_MODEL_NAME must equal the deployed model's name (e.g., `gpt-4o`)

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# Note: We use the async version of DefaultAzureCredential for the Agent Framework
from azure.identity.aio import DefaultAzureCredential

## 6.2. Creating the Research Assistant Agent

### 6.2.1 Import Required Libraries

Import the Agent Framework components and Azure Identity for authentication.

In [3]:
from agent_framework.azure import AzureAIClient
from agent_framework import MCPStreamableHTTPTool

### 6.2.2 Define the MCP Tool

Create a function that returns the MCP tool configuration for Microsoft Learn documentation. This allows your agent to query live, up-to-date documentation.

In [4]:
def create_mcp_tools():
    """Create MCP tools for the Research Assistant agent."""
    return [
        MCPStreamableHTTPTool(
            name="Microsoft Learn MCP",
            description="Provides trusted, up-to-date information from Microsoft's official documentation",
            url="https://learn.microsoft.com/api/mcp",
        )
    ]

### 6.2.3 Define the Agent Instructions

Create the system instructions that define how the Research Assistant should behave.

In [5]:
AGENT_INSTRUCTIONS = """
You are a helpful research assistant that specializes in Azure and Microsoft technologies. 

Your responsibilities:
1. Use the Microsoft Learn MCP tool to find accurate, up-to-date documentation when answering questions
2. Always cite your sources by providing links to the documentation
3. If you're unsure about something, acknowledge it and suggest where the user might find more information
4. Provide clear, concise explanations suitable for developers of varying experience levels

When responding:
- Start with a direct answer to the question
- Provide relevant code examples when appropriate
- Include links to official documentation for further reading
"""

### 6.2.4 Set Up Environment Variables

Load the project endpoint and model deployment from your `.env` file.

In [ ]:
PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT", "").strip()
assert PROJECT_ENDPOINT, "ERROR: AZURE_AI_PROJECT_ENDPOINT is missing"

MODEL_DEPLOYMENT = os.getenv("CHAT_MODEL_NAME", "").strip()
assert MODEL_DEPLOYMENT, "ERROR: CHAT_MODEL_NAME is missing"

print(f"Project Endpoint: {PROJECT_ENDPOINT}")
print(f"Model Deployment: {MODEL_DEPLOYMENT}")

## 6.3. Testing the Research Assistant

### 6.3.1 Single Query Test

Let's test the agent with a single question about Azure services.

In [ ]:
async def ask_agent(question: str):
    """Send a single question to the Research Assistant agent."""
    async with (
        DefaultAzureCredential() as credential,
        AzureAIClient(
            project_endpoint=PROJECT_ENDPOINT,
            model_deployment_name=MODEL_DEPLOYMENT,
            credential=credential,
        ).as_agent(
            name="ResearchAssistant",
            instructions=AGENT_INSTRUCTIONS,
            tools=create_mcp_tools(),
        ) as agent,
    ):
        print(f"Question: {question}\n")
        print("Assistant: ", end="", flush=True)
        
        async for chunk in agent.run(question, stream=True):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

# Test with a sample question
await ask_agent("What is Azure Kubernetes Service and when should I use it?")

### 6.3.2 Multi-Turn Conversation with Thread

One of the powerful features of the Agent Framework is thread persistence, which maintains context across multiple conversation turns.

In [ ]:
async def multi_turn_conversation(questions: list):
    """Demonstrate multi-turn conversation with context retention."""
    async with (
        DefaultAzureCredential() as credential,
        AzureAIClient(
            project_endpoint=PROJECT_ENDPOINT,
            model_deployment_name=MODEL_DEPLOYMENT,
            credential=credential,
        ).as_agent(
            name="ResearchAssistant",
            instructions=AGENT_INSTRUCTIONS,
            tools=create_mcp_tools(),
        ) as agent,
    ):
        # Create a session for multi-turn conversation
        session = agent.create_session()
        
        for i, question in enumerate(questions, 1):
            print(f"--- Turn {i} ---")
            print(f"You: {question}\n")
            print("Assistant: ", end="", flush=True)
            
            async for chunk in agent.run(question, stream=True, session=session):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n")

# Test multi-turn conversation
questions = [
    "How do I set up managed identity for an Azure Function?",
    "Can you show me a code example for that?",
    "What are the security benefits of using managed identity instead of connection strings?"
]

await multi_turn_conversation(questions)

## 6.4. Exploring Agent Capabilities

### 6.4.1 Adding Custom Tools

In addition to MCP tools, you can create custom Python functions as tools. Here's an example of adding a simple calculation tool.

In [ ]:
from typing import Annotated

def calculate_azure_storage_cost(
    storage_gb: Annotated[float, "Amount of storage in GB"],
    tier: Annotated[str, "Storage tier: 'hot', 'cool', or 'archive'"] = "hot"
) -> str:
    """Calculate estimated monthly cost for Azure Blob Storage."""
    # Simplified pricing (actual prices vary by region)
    prices = {
        "hot": 0.0184,
        "cool": 0.01,
        "archive": 0.00099
    }
    price_per_gb = prices.get(tier.lower(), prices["hot"])
    monthly_cost = storage_gb * price_per_gb
    return f"Estimated monthly cost for {storage_gb} GB on {tier} tier: ${monthly_cost:.2f}"

# You can add this tool to your agent like this:
# tools=[get_mcp_tools()[0], calculate_azure_storage_cost]

### 6.4.2 Try It Yourself!

Use the cell below to ask your own questions to the Research Assistant. Modify the question and run the cell to see the response.

In [ ]:
# Try your own question!
your_question = "What are the best practices for Azure OpenAI prompt engineering?"

await ask_agent(your_question)

### 6.4.3 Try this in the new Foundry Portal (optional)

The Microsoft Foundry portal also provides a no-code experience for creating and testing agents. If you'd like to explore the portal-based approach:

1. Navigate to [Microsoft Foundry](https://ai.azure.com) and open your project using the New Foundry portal 
2. Click **Build** in the top right
3. If you already did the steps above, you should already see a ResearchAssistant and you can click that. Otherwise, create a new agent and give it a name like "ResearchAssistant"
4. Add instructions similar to what we defined in `AGENT_INSTRUCTIONS` above
5. Under **Tools**, add the Microsoft Learn MCP tool to give your agent access to documentation
6. Use the **Playground** to test your agent with the same questions you tried in this notebook

Compare the portal experience with the code-first approach you used here. Consider:
- When would you prefer the portal vs. code?
- How might you use both together in a development workflow?

## 6.5. Summary

In this notebook, you learned how to:

1. **Set up the Microsoft Agent Framework** with the `agent-framework-azure-ai` package
2. **Create MCP tools** to connect your agent to live data sources (Microsoft Learn)
3. **Build a Research Assistant agent** with custom instructions
4. **Use thread persistence** for multi-turn conversations
5. **Extend agents with custom tools** using Python functions

### Next Steps

Consider exploring:
- Adding more MCP tools (e.g., GitHub, databases)
- Creating multi-agent systems for complex workflows
- Implementing agent handoffs for specialized tasks
- Adding memory and state management for long-running agents